In [1]:
import pandas as pd
import numpy as np
#from ipyleaflet import AwesomeIcon, Marker, Map, LegendControl, MarkerCluster, ScaleControl, FullScreenControl, LayersControl

import folium
from folium import plugins
from branca.element import Template, MacroElement
import geocoder

import ipywidgets as widgets
from IPython.display import display

from ast import literal_eval

import warnings
warnings.filterwarnings("ignore")

In [2]:
unsc = pd.read_excel("cleaned_data_unsc.xlsx")
locs = pd.read_excel("country_locations.xlsx")
chatgpt = pd.read_excel("chatgpt.xlsx", sheet_name = "output")
unsc = unsc.merge(chatgpt, on = "Meeting Number", how = "left")

In [3]:
#Fixing Datatypes
#In unsc: the process of saving the dataframe from the previous notebook which cleaned the data  and reading it back in has led to the lists being read in as strings, convert them back to lists here
#Same for tuples being read in as str for locs

In [4]:
unsc["Submitted By"] = unsc.apply(lambda r: literal_eval(r["Submitted By"]), axis = 1)
unsc["In Favor"] = unsc.apply(lambda r: literal_eval(r["In Favor"]), axis = 1)
unsc["Abstain"] = unsc.apply(lambda r: literal_eval(r["Abstain"]), axis = 1)
unsc["Against"] = unsc.apply(lambda r: literal_eval(r["Against"]), axis = 1)

In [5]:
locs["locations"] = locs.apply(lambda r: literal_eval(r["locations"]), axis = 1)

In [6]:
def display_vote(input_df):
    """format this string so I can print it with the map."""
    str_to_print = ""
    for category in ["Submitted By", "In Favor", "Abstain", "Against"]:
        relevant_list = input_df[category].values.tolist()[0]
        if len(relevant_list) > 0:
            str_to_print = str_to_print + "\n\n" + category + ": "
            for country in relevant_list:
                if str_to_print == "":
                    str_to_print = country
                else:
                    str_to_print = str_to_print + ", " + country
    str_to_print = str_to_print.replace(" , ", " ")
    print(str_to_print)

In [7]:
widget_width = "250px"
padding = "0px 0px 0px 4px"

In [8]:
def unsc_to_mappable(unsc_df, voting_on_from_widget = "Meeting #: 381 Voting On: S/1076 Last Paragraph"):
    df_to_map_og = unsc[unsc["Meeting Number: Voting On"] == voting_on_from_widget]
    
    df_to_map = df_to_map_og[["Voting On", "Meeting Number", "Submitted By", "In Favor", "Abstain", "Against"]]
    sub_by = df_to_map["Submitted By"].values[0][0]
    df_to_map["[Meeting] Document"] = str(df_to_map["Meeting Number"].values) + " " + str(df_to_map["Voting On"].values)
    df_to_map = pd.melt(df_to_map, id_vars = "[Meeting] Document", value_vars = ["In Favor", "Abstain", "Against"], var_name = "Vote", value_name = "Country")
    df_to_map = df_to_map.explode("Country")
    df_to_map = df_to_map[df_to_map["Country"].notnull()]

    
    df_to_map["Is_submitter"] = df_to_map.apply(lambda r: "(Submitted by) " if r["Country"] == sub_by
                                                                    else "", axis = 1)
    #icon from how the country voted
    df_to_map["Icon1"] = df_to_map["Vote"].map({"In Favor": "glyphicon-ok-circle", 
                                                          "Abstain":"glyphicon-ban-circle",
                                                         "Against":"glyphicon-remove-circle"})
    #icon from if the country submitted the resolution
    df_to_map["Icon2"] = df_to_map["Is_submitter"].map({"(Submitted by) ": "glyphicon-edit",
                                                                               "": None})
    #icon determined by Icon2 for submitter only, else Icon1
    df_to_map["Icon to Use"] = df_to_map.apply(lambda r: r["Icon2"] if r["Is_submitter"] != "" else r["Icon1"], axis = 1)
    #color from how the country voted
    df_to_map["Color"] = df_to_map["Vote"].map({"In Favor": "green", "Abstain":"gray", "Against": "red"}) 
    
    df_to_map["Country:Vote"] = df_to_map["Is_submitter"] + df_to_map["Country"] + ": " + df_to_map["Vote"]
    my_map = folium.Map(location = [10,30], zoom_start = 2)
    df_to_map.apply(lambda row: folium.Marker(location = [locs[locs["country_name"] == row["Country"]]["locations"].values[0][0], 
                                                          locs[locs["country_name"] == row["Country"]]["locations"].values[0][1]],
                                            popup = row["Country:Vote"], icon = folium.Icon(color = row["Color"],
                                            icon = row["Icon to Use"], prefix = "glyphicon")).add_to(my_map), axis = 1)
    
    # add mini reference map
    my_map.add_child(plugins.MiniMap(toggle_display=True, position = "bottomleft"))
    
    #add full screen button to map
    plugins.Fullscreen(position = "topright").add_to(my_map)
    
    #add layer control 
    #folium.LayerControl().add_to(my_map)
    
    #add legend
    my_template = """
    {% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Voted: </div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:#68a023;opacity:0.7;'></span>In Favor</li>
        <li><span style='background:#565656;opacity:0.7;'></span>Abstain</li>
        <li><span style='background:#ce3c28;opacity:0.7;'></span>Against</li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""

    my_macro = MacroElement()
    my_macro._template = Template(my_template)

    my_map.get_root().add_child(my_macro)

    title_text = "Voting Results on: " + str(df_to_map_og["Voting On"].values[0])
    title_html = '''
             <h1 align="center" style="font-size:25px"><b>{}</b></h1>
             '''.format(title_text)   
    
    my_map.get_root().html.add_child(folium.Element(title_html))
    
    subtitle_text = "Year: " + str(df_to_map_og["Year"].values[0]) + ", " + "Meeting Number:" + str(df_to_map_og["Meeting Number"].values[0]) 
    
    subtitle_html = '''
                <h2 align="center" style = "font-size:20px"><b>{}</b></h2>
                '''.format(subtitle_text)
    
    my_map.get_root().html.add_child(folium.Element(subtitle_html))
    
    print("Meeting Year:", df_to_map_og["Year"].values[0], "\n")
    print("Meeting Subject:", df_to_map_og["Topic Name"].values[0], "\n")
    print("Meeting Summary:", df_to_map_og["Summary"].values[0])
    
    #display(my_map)
    
    display_vote(df_to_map_og)
    return my_map
    #have an option to reset the query
#     reset_query_button = widgets.Button(
#         description = "Clear Search",
#         disabled = False,
#         button_style = "danger",
#         layout = widgets.Layout(width = widget_width, padding = padding)
#     )
    
#     reset_query_

In [9]:
def find_uid(df, my_year, my_topic = None, my_doc = None):
    """Get a unique identifier to identify the parameter that is necessary for the function to visualize/map the data."""
    df = df[df["Year"] == my_year]
    if my_topic is not None: 
        df = df[df["Topic Name"] == my_topic]
    if my_doc is not None: 
        df = df[df["Voting On"] == my_doc]
        
    assert len(df["Meeting Number: Voting On"].values.tolist()) == 1
    return df["Meeting Number: Voting On"].values[0]

In [10]:
def interactive_part():
    year_options = sorted(unsc["Year"].unique().tolist())

    yr_dropdown = widgets.Dropdown(
        options = year_options,
        value = 1947,
        description = "Year:",
        layout = widgets.Layout(width = widget_width, padding = padding),
        style = {"description_width": "initial"}
    )

    yr_search = widgets.Button(
        description = "Search",
        button_style = "success",
        disabled = False,
        icon = "check",
        layout = widgets.Layout(width = widget_width, padding = padding)

    )

    reset_button = widgets.Button(
        description = "Reset",
        button_style = "primary",
        disabled = False,
        icon = "refresh",
        layout = widgets.Layout(width = widget_width, padding = padding)

    )

    # define all the output areas here so in the interactions below they can 
    # be manipulated (with output: show something, or clear the output)
    #yr_output = widgets.Output()
    #topic_output = widgets.Output()
    #document_options_output = widgets.Output()    
    map_output = widgets.Output()

    # ==================================================================
    # layout widget and app content
    # ==================================================================
    # The app layout will be as follows -- created in the code at the end of the 
    # controls initialized to be a VBox with year dropdown and yr_resarch
    # controls will be updated during the interation
    # controls is going to be:
    # ----------------------------------
    # | widget one | widget two | .... |
    # ----------------------------------
    
    # intially, only show yr_dropdown and yr_search
    #   in the interation code below, modify the children of controls to 
    controls = widgets.HBox([yr_dropdown, yr_search])
    
    # all_app is going to be:
    # ------------------------------
    # |  controls                  |
    # ------------------------------
    # |  map_output                |
    # ------------------------------
    #all_app = widgets.HBox([controls, yr_output, topic_output, document_options_output, map_output])
    all_app = widgets.VBox([controls, map_output])

    def clear_map():
        with map_output:
            map_output.clear_output()

    def reset_everything(btn=None):
        map_output.clear_output()
        controls.children = [yr_dropdown, yr_search]

    reset_button.on_click(reset_everything)

    #functionality
    def search1(look):
        yr_selected = yr_dropdown.value

        yr_subset = unsc[unsc["Year"] == yr_selected]
        document_select_options = list(set(unsc[unsc["Year"] == yr_selected]["Voting On"].values.tolist()))

        # if there is only one document for that year, directly visualize
        if len(document_select_options) == 1:
            param_to_map = find_uid(unsc, yr_selected)
            # display map to map_output
            controls.children = [yr_dropdown, yr_search]
            #reset_everything()
            with map_output:
                map_output.clear_output()
                display(unsc_to_mappable(unsc, param_to_map))

        else:
            clear_map()
            yr_subset = unsc[unsc["Year"] == yr_selected]
            
            # build topic_options
            document_select_options = [thing for thing in list(set(unsc[unsc["Year"] == yr_selected]["Topic Name"].values.tolist())) if pd.notnull(thing)]
            topic_options = widgets.RadioButtons(
                options = document_select_options,
                value = None,
                description = "Topic of Interest (" + str(yr_selected) + "):",
                style = {"description_width": "initial"},
                layout = {"width": "max-content"}
            )

            topic_submit = widgets.Button(
                description = "Search",
                disabled=False,
                button_style = "Success",
                icon = "check",
                layout = widgets.Layout(width = widget_width, padding = padding)
            )

            # update controls
            controls.children = [topic_options, topic_submit, reset_button]

            def search2(look2):
                topic_chosen = topic_options.get_interact_value()
                yr_topic = yr_subset[yr_subset["Topic Name"] == topic_chosen]

                if len(yr_topic) == 1:
                    param_to_map = find_uid(unsc, yr_selected, topic_chosen)
                    with map_output:
                        map_output.clear_output()
                        display(unsc_to_mappable(unsc, param_to_map))
                elif len(yr_topic) > 1:
                    clear_map()
                    document_options = [doc for doc in yr_topic["Voting On"].unique().tolist() if pd.notnull(doc)]
                    document_options_wid = widgets.RadioButtons(
                        options = document_options,
                        value = None,
                        description = "Document of Interest (" + topic_chosen + "): ",
                        style = {"description_width": "initial"},
                        layout = {"width": "max-content"})

                    document_options_submit = widgets.Button(
                        description = "Visualize",
                        disabled=False,
                        icon = "globe",
                        button_style = "success", # 'success', 'info', 'warning', 'danger' or ''
                        style = {"description_width": "initial"})
                            
                    # update controls here
                    controls.children = [document_options_wid, document_options_submit, reset_button]
                        
                    #third layer of functionality
                    def search3(look3):
                        document_chosen = document_options_wid.get_interact_value()
                        document_df = yr_topic[yr_topic["Voting On"] == document_chosen]
                        param_to_map = find_uid(unsc, yr_selected, topic_chosen, document_chosen)
                        with map_output:
                            map_output.clear_output()
                            display(unsc_to_mappable(unsc, param_to_map))

                    document_options_submit.on_click(search3)

            #display(topic_box)
            topic_submit.on_click(search2)

    yr_search.on_click(search1)

    # show the app
    display(all_app)
    #search1(None)


interactive_part()